In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter, defaultdict
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
import lightgbm as lgb
from functools import partial
import copy
import time
import matplotlib.pyplot as plt
import scipy as sp
import random
from hyperopt import hp, tpe, Trials, fmin, space_eval
import cv2
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train = pd.read_csv("../../../20200125atma/input/train.csv")
test = pd.read_csv("../../../20200125atma/input/test.csv")
userlog = pd.read_csv("../../../20200125atma/input/user_log.csv")
poi = pd.read_csv("../../../20200125atma/input/poi.csv")

In [3]:
list_y = []
for i in range(13411):
    img = cv2.imread('../../../20200125atma/input/images/'+str(i)+'.png')
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  # HSV 色空間に変換

    red = cv2.inRange(hsv, np.array([145, 70, 0]), np.array([180, 255, 255]))
    yellow = cv2.inRange(hsv, (15,0,0), (36, 255, 255))
    green = cv2.inRange(hsv, np.array([30, 190, 0]), np.array([90, 255, 255]))
    blue = cv2.inRange(hsv, np.array([108, 121, 0]), np.array([120, 255, 255]))
    white = cv2.inRange(hsv, np.array([108, 21, 0]), np.array([255, 70, 255]))

    # 白だけゴミがあるので、収縮演算
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    white = cv2.erode(yellow, kernel)

    bin_imgs = {'red': red, 'yellow': yellow, 'green': green,
            'blue': blue, 'white': white}

    for label, bin_img in bin_imgs.items():
        contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = list(filter(lambda cnt: len(cnt) > 30, contours))
        count = len(contours)
    
        if label == "yellow":
            list_y.append(count)

In [4]:
def preprocess(train, test, userlog, poi):
    userlog["sysname"][userlog.sysname == "ANDROID"] = "Android"
    userlog["lang"][userlog.lang == "ja_JP"] = "ja_JP"
    poi = poi.rename(columns={"latitude": "store_lat", "longitude": "store_lon"})
    train = pd.merge(train, poi, on ="pid", how = "left")
    test = pd.merge(test, poi, on ="pid", how = "left")
    userlog = pd.merge(userlog, train[["session_id", "store_lat", "store_lon", "radius", "pid"]], on="session_id", how="left")
    userlog = pd.merge(userlog, test[["session_id", "store_lat", "store_lon", "radius", "pid"]], on="session_id", how="left")
    userlog["store_lat"] = np.nanmax(userlog[["store_lat_x", "store_lat_y"]], axis=1)
    userlog["store_lon"] = np.nanmax(userlog[["store_lon_x", "store_lon_y"]], axis=1)
    userlog["radius"] = np.nanmax(userlog[["radius_x", "radius_y"]], axis=1)
    userlog["pid"] = np.nanmax(userlog[["pid_x", "pid_y"]], axis=1)
    drop_features = ["store_lat_x", "store_lat_y", "store_lon_x", "store_lon_y", "radius_x", "radius_y", "pid_x", "pid_y"]
    userlog.drop(drop_features, axis=1, inplace=True)
    userlog["distance"] = np.sqrt((userlog["latitude"]- userlog["store_lat"])**2 + (userlog["longitude"]-userlog["store_lon"])** 2 )
    userlog["time"] = userlog["hour"].map(str) + str(":") + userlog["minute"].map(str) + str(":") + userlog["second"].map(str)
    userlog["time"] = pd.to_datetime(userlog['time'],format= '%H:%M:%S' )
    userlog = userlog.sort_values(["session_id", "hour", "minute", "second"]).reset_index(drop=True)
    userlog["virtual_dis"] = 6370 * np.arccos(np.sin(userlog["latitude"])*np.sin(userlog["store_lat"]) + np.cos(userlog["latitude"])*np.cos(userlog["store_lat"])*np.cos(userlog["longitude"]-userlog["store_lon"]))
    userlog["in_store"] = userlog["virtual_dis"] < userlog["radius"]
    unique_sess = pd.DataFrame(userlog.groupby("pid")["session_id"].nunique().copy().reset_index(drop=False))
    unique_sess = unique_sess.rename(columns = {"session_id": "uni_sess_per_pid"})
    userlog = pd.merge(userlog, unique_sess, on ="pid", how = "left")
    return train, test, userlog, poi, unique_sess
train, test, userlog, poi, unique_sess = preprocess(train, test, userlog, poi)

In [5]:
def encode(train, test, userlog):
    os_list =  sorted(list(set(userlog['sysname'].unique())))
    os_map = dict(zip(os_list, np.arange(len(os_list))))
    userlog["sysname"] = userlog["sysname"].map(os_map)
    
    lang_list =  sorted(list(set(userlog['lang'].unique())))
    lang_map = dict(zip(lang_list, np.arange(len(lang_list))))
    userlog["lang"] = userlog["lang"].map(lang_map)
    
    timezone_list =  sorted(list(set(userlog['timezone'].unique())))
    timezone_map = dict(zip(timezone_list, np.arange(len(timezone_list))))
    userlog["timezone"] = userlog["timezone"].map(timezone_map)

    return train, test, userlog
train, test, userlog = encode(train, test, userlog)

In [6]:
userlog.head(3)

,latitude,longitude,sysname,optout,lang,timezone,session_id,hour,minute,second,day_of_week,categorical_1,categorical_2,categorical_3,categorical_4,categorical_5,categorical_6,store_lat,store_lon,radius,pid,distance,time,virtual_dis,in_store,uni_sess_per_pid
0,35.650300,46.780509,0,0,9,1,0003f26df5d8b928b416fba58efd5c91,0,33,31,6,4,0,44,1.0,1,187,35.651739,46.776134,118.0,354655367.0,0.004606,1900-01-01 00:33:31,15.744307,True,16
1,35.650298,46.780512,0,0,9,1,0003f26df5d8b928b416fba58efd5c91,0,33,31,6,4,1,44,1.0,1,187,35.651739,46.776134,118.0,354655367.0,0.004609,1900-01-01 00:33:31,15.758882,True,16
2,35.650286,46.780524,0,0,9,1,0003f26df5d8b928b416fba58efd5c91,2,18,33,6,4,2,44,1.0,1,187,35.651739,46.776134,118.0,354655367.0,0.004624,1900-01-01 02:18:33,15.832117,True,16


In [7]:
def get_logdata(user_sample):
    day_of_week_counts = {"day"+str(day) : 0 for day in range(7)}

    all_results = []
    features = {"accumulated_actions":0}
    features["accumulated_actions"] = user_sample.shape[0]
    features["session_id"] = user_sample.iloc[0]["session_id"]
    features["OS"] = user_sample.iloc[0]["sysname"]
    features["lang"] = user_sample.iloc[0]["lang"]
    features["timezone"] = user_sample.iloc[0]["timezone"]
    features["optout_count"] = np.sum(user_sample["optout"])
    distance = np.array(user_sample["distance"])
    features["max_dist"] = np.nanmax(distance)
    features["min_dist"] = np.nanmin(distance)
    features["std_dist"] = np.nanstd(distance)
    features["closest_lat"] = user_sample[user_sample.distance == features["min_dist"]].iloc[0]["latitude"]
    features["closest_lon"] = user_sample[user_sample.distance == features["min_dist"]].iloc[0]["longitude"]
    features["categorical_1"] = user_sample["categorical_1"].nunique()
    features["categorical_2"] = user_sample["categorical_2"].nunique()
    features["categorical_3"] = user_sample["categorical_3"].nunique()
    features["categorical_4"] = user_sample["categorical_4"].nunique()
    features["categorical_5"] = user_sample["categorical_5"].nunique()
    features["categorical_6"] = user_sample["categorical_6"].nunique()
    #features["closest_hour"] = user_sample[user_sample.distance == features["min_dist"]].iloc[0]["hour"]
    #features["closest_min"] = user_sample[user_sample.distance == features["min_dist"]].iloc[0]["minute"]
    #features["closest_second"] = user_sample[user_sample.distance == features["min_dist"]].iloc[0]["second"]
    #instore_queue = np.array2string(np.array(user_sample["in_store"].astype(int)), separator=',')
    #features["exit_count"] = instore_queue[1:-1].replace(',', '').count("10")
    
    n_of_days = Counter(user_sample['day_of_week']) 
    for key in n_of_days.keys():
        day_of_week_counts["day"+str(key)] += n_of_days[key]
    features.update(day_of_week_counts)
    
    features["in_store"] = np.sum(user_sample["in_store"])
    
    all_results.append(features)
    return all_results

In [8]:
def get_log_info(userlog):
    compiled_log = []

    for i, (ses_id, user_sample) in tqdm(enumerate(userlog.groupby('session_id', sort=False)), total=userlog.session_id.nunique(), desc='session_id', position=0):
        compiled_log += get_logdata(user_sample)
    reduced_log = pd.DataFrame(compiled_log)
    return reduced_log 
reduced_log = get_log_info(userlog)

In [9]:
def postprocess(train, test, reduced_log, unique_sess):
    drop_features = ["imid", "pid"]
    new_train = pd.merge(train, reduced_log, on ="session_id", how = "left")
    new_test = pd.merge(test, reduced_log, on ="session_id", how = "left")
    new_train["yellow_in_pic"] = list_y[:6612]
    new_test["yellow_in_pic"] = list_y[6612:]
    pid_count_mean = train.groupby('pid').target.count()
    new_train['pid_count_enc'] = new_train['pid'].map(pid_count_mean)
    new_test['pid_count_enc'] = new_test['pid'].map(pid_count_mean)
    new_train = pd.merge(new_train, unique_sess, on ="pid", how = "left")
    new_test = pd.merge(new_test, unique_sess, on ="pid", how = "left")
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(new_train["session_id"]))
    new_train["session_id"] = lbl.transform(list(new_train["session_id"]))
    new_train.drop(drop_features, axis=1, inplace=True)
    new_test.drop(drop_features, axis=1, inplace=True)
    print(new_train.shape, new_test.shape)
    return new_train, new_test
new_train, new_test = postprocess(train, test, reduced_log, unique_sess)

(6612, 34) (6799, 33)


In [10]:
new_train.head(3)

,target,session_id,store_lat,store_lon,radius,type,name,OS,accumulated_actions,categorical_1,categorical_2,categorical_3,categorical_4,categorical_5,categorical_6,closest_lat,closest_lon,day0,day1,day2,day3,day4,day5,day6,in_store,lang,max_dist,min_dist,optout_count,std_dist,timezone,yellow_in_pic,pid_count_enc,uni_sess_per_pid
0,0,2315,36.320751,46.104755,25,2,0,1,187,2,3,2,1,4,7,36.320923,46.104648,0,0,0,0,187,0,0,149,7,0.072056,0.000203,0,0.014947,1,4,15,26
1,0,2279,35.564913,46.744633,51,0,1,0,111,1,3,1,1,2,18,35.565118,46.743960,111,0,0,0,0,0,0,111,9,0.005153,0.000704,0,0.000580,1,3,18,37
2,0,959,35.693777,46.784288,189,0,1,0,262,1,3,1,0,3,1,35.693449,46.784401,0,0,0,0,0,0,262,262,9,0.014393,0.000347,0,0.004819,1,1,11,21


In [58]:
categoricals = ['lang', 'OS']

lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','learning_rate': 0.017891320270412462,'max_depth': 5, 'random_seed':42}
    
lgbm_params2 = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial',   'learning_rate': 0.01861754102536491,'random_seed':43, 'max_depth': 4}

lgbm_params3 = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial', 'learning_rate': 0.09482045811322946, 'random_seed':44,'max_depth': 3}

def modelling(new_train, new_test, lgbm_params):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    remove_features = ["session_id", "categorical_1", "categorical_2", "categorical_3"]
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds)
    models = []
    
    valid = np.array([])
    real = np.array([])
    evals_result = {}
    features_list = [i for i in X_train.columns]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    initial = lgbm_params["random_seed"]
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        #lgbm_params["random_seed"] = initial + i
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]

        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        #clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
        #   num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500, categorical_feature = categoricals)
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],num_boost_round=350, categorical_feature = categoricals, verbose_eval = 100)
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        valid = np.concatenate([valid, valid_predict])
        real = np.concatenate([real, y_test2])
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()

        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

    roc = roc_auc_score(real, valid)
    print("ROC = {}".format(roc_auc_score(real, valid)))
    print(confusion_matrix(real, np.round(valid)))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return roc, pred_value, feature_importance_df
    
#roc1, pred_value, _ = modelling(new_train, new_test, lgbm_params)
roc2, pred_value2, feature_importance_df = modelling(new_train, new_test, lgbm_params2)
#roc3, pred_value3, _ = modelling(new_train, new_test, lgbm_params3)

Fold 1
[100]	training's auc: 0.97963	valid_1's auc: 0.873342
[200]	training's auc: 0.992427	valid_1's auc: 0.868999
[300]	training's auc: 0.996515	valid_1's auc: 0.857253
Fold 2
[100]	training's auc: 0.978911	valid_1's auc: 0.848408
[200]	training's auc: 0.987946	valid_1's auc: 0.873714
[300]	training's auc: 0.993456	valid_1's auc: 0.878858
Fold 3
[100]	training's auc: 0.978445	valid_1's auc: 0.82023
[200]	training's auc: 0.993717	valid_1's auc: 0.834491
[300]	training's auc: 0.997144	valid_1's auc: 0.839249
Fold 4
[100]	training's auc: 0.984065	valid_1's auc: 0.831568
[200]	training's auc: 0.994129	valid_1's auc: 0.840426
[300]	training's auc: 0.997784	valid_1's auc: 0.844284
Fold 5
[100]	training's auc: 0.979362	valid_1's auc: 0.834838
[200]	training's auc: 0.990711	valid_1's auc: 0.86044
[300]	training's auc: 0.995023	valid_1's auc: 0.871636
ROC = 0.8572531034330694
[[6472    7]
 [ 122   11]]


In [29]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices
        
def modelling_sgk(new_train, new_test, lgbm_params):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    remove_features = []
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)
    groups = np.array(X_train.session_id.values)
    X_test = X_test.drop("session_id", axis=1)
    models = []
    
    n_folds = 5
    valid = np.array([])
    real = np.array([])
    evals_result = {}
    features_list = [i for i in X_train.columns if i != "session_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    for i, (train_index, test_index) in enumerate(stratified_group_k_fold(X_train, y_train, groups, k=n_folds, seed=12)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2.drop("session_id", axis=1, inplace=True)

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        X_test2.drop("session_id", axis=1, inplace=True)

        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
            num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500, categorical_feature = categoricals)
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        valid = np.concatenate([valid, valid_predict])
        real = np.concatenate([real, y_test2])
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()

        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

    roc = roc_auc_score(real, valid)
    print("ROC = {}".format(roc_auc_score(real, valid)))
    print(confusion_matrix(real, np.round(valid)))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return roc, pred_value, feature_importance_df
    
#roc_sgk1, pred_value, _ = modelling_sgk(new_train, new_test, lgbm_params)
#roc_sgk2, pred_value2, _ = modelling_sgk(new_train, new_test, lgbm_params2)
#roc_sgk3, pred_value3, _ = modelling_sgk(new_train, new_test, lgbm_params3)

In [59]:
feature_importance_df.sort_values("Cv", ascending = True).reset_index(drop=True)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,min_dist,504,599,583,548,487,544.2,43.383868,0.079720
1,accumulated_actions,243,267,259,309,302,276.0,25.392912,0.092003
2,radius,310,361,424,356,396,369.4,38.645051,0.104616
3,in_store,215,225,254,247,289,246.0,25.752670,0.104686
4,categorical_6,108,103,94,131,124,112.0,13.608821,0.121507
5,day2,65,88,59,70,60,68.4,10.556515,0.154335
6,std_dist,213,278,324,311,227,270.6,44.175106,0.163249
7,store_lat,158,210,231,249,177,205.0,33.555923,0.163687
8,day1,72,55,42,63,61,58.6,9.931767,0.169484
9,closest_lat,139,148,189,166,222,172.8,29.955968,0.173356


In [31]:
def my_hyperopt(X, Y):
    def para_tuning_obj(params):
        params = {
        'boosting_type': 'gbdt', 
        'metric': 'auc', 
        'objective': 'binary', 
        'eval_metric': 'auc', 
        "tree_learner": "serial",
        'max_depth': 3,
        'learning_rate': float(params['learning_rate']),
}
    
        real = np.array([])
        pred = np.array([])
        skf = StratifiedKFold(n_splits=5)
        for trn_idx, val_idx in skf.split(X, Y):
            x_train, x_val = X.iloc[trn_idx, :], X.iloc[val_idx, :]
            y_train, y_val = Y.iloc[trn_idx], Y.iloc[val_idx]
            train_set = lgb.Dataset(x_train, y_train)
            val_set = lgb.Dataset(x_val, y_val)
        
            clf = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                         valid_sets = [train_set, val_set], verbose_eval = 300)
            pred = np.concatenate((pred, np.array(clf.predict(x_val, num_iteration = clf.best_iteration))), axis=0) 
            real = np.concatenate((real, np.array(y_val)), axis=0) 
        score = roc_auc_score(real, pred)
    
        return -score

    trials = Trials()

    space ={
        'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
    }

    best = fmin(para_tuning_obj, space = space, algo=tpe.suggest, max_evals=10, trials=trials, verbose=1)

    best_params = space_eval(space, best)
    return best_params

#X = new_train.drop(['target'],axis=1).copy()
#Y = new_train.target.copy()
#my_hyperopt(X, Y)

In [60]:
print(new_train.columns)

Index(['target', 'session_id', 'store_lat', 'store_lon', 'radius', 'type',
       'name', 'OS', 'accumulated_actions', 'categorical_1', 'categorical_2',
       'categorical_3', 'categorical_4', 'categorical_5', 'categorical_6',
       'closest_lat', 'closest_lon', 'day0', 'day1', 'day2', 'day3', 'day4',
       'day5', 'day6', 'in_store', 'lang', 'max_dist', 'min_dist',
       'optout_count', 'std_dist', 'timezone', 'yellow_in_pic',
       'pid_count_enc', 'uni_sess_per_pid'],
      dtype='object')


In [61]:
final_pred = pred_value2
roc = roc2
sample_submission = pd.read_csv("../../../20200125atma/input/atmacup3_sample_submission.csv")
sample_submission["target"] = final_pred
sample_submission.to_csv("../../../20200125atma/result/atmacup3_sample_submission"+str(roc)+".csv", index = False)

- quit early stopping and adapt fixed epoch